# The HCIPy segmented mirror on non-normalized apertures an realistic wavelengths

In this notebook I am making my segmented mirror class for HCIPy work also for non-normalized apertures and wavelengths other than 1 (m).